In [1]:
# Load the gams extension
%load_ext gams_magic

# Support Vector Machines}

Support vector machines are a popular method for classification
within the machine learning community.  Essentially, a linear
  classifier $f$ is generated as $f(x) = w'x + \gamma$ such that if
  $f(x) > 0$ then $x$ is in class $P_+$ whereas if $f(x) < 0$ then $x
  \in P_-$.
  
Once you have the classifier, you can use it with new data $x$ to predict if
$x \in P_+$ or $x \in P_i$ just by evaluating $f(x)$.

In this exercise, you should use the data that is provided in the abalone.gdx file,
and you should attempt to predict whether the ``number of rings'' is greater than 10 or not.  The data file provides 4177 samples, with the following features (including the 
number of rings):

|Name           |Data Type      |Meas.  |Description                |
|---------------|---------------|-------|---------------------------|
|Sex            |nominal        |       |M, F, and I (infant)       |
|Length         |continuous     |mm     |Longest shell measurement  |
|Diameter       |continuous     |mm     |perpendicular to length    |
|Height         |continuous     |mm     |with meat in shell         |
|Whole weight   |continuous     |grams  |whole abalone              |
|Shucked weight |continuous     |grams  |weight of meat             |
|Viscera weight |continuous     |grams  |gut weight (after bleeding)|
|Shell weight   |continuous     |grams  |after being dried          |
|Rings          |integer        |       |+1.5 gives the age in years|
 
Note that the first nominal value has been converted to 3 binary
values and can be treated as separate data for this assignment.
The following gams code uses the set x to index the features, excluding the "rings" that is
stored as a label y indicating whether the number of rings is greater than 10 or not.

Instead of using all the data to generate the classifier $f$, we use a subset of the samples $i$ as training data, another set as tuning data to determine some tradeoff parameters and the remaining data as the testing data (to see how well we do).  You should pretend that you do not know the values of y for the testing data since normally you won't!

In [2]:
%%gams
set i(*), headr(*);
parameter Data(i,headr);
$gdxin abalone.gdx
$LOAD i
$LOAD headr
$LOAD Data
$gdxin

set  x(headr)  index of independent variables;
x(headr) = yes$(not sameas(headr,'Rings'));

parameter y(i);
y(i) = -1 + 2$(Data(i,'Rings') gt 10);

set train(i), tuning(i), test(i);
train(i) = yes$(ord(i) le 3000);
tuning(i) = yes$(ord(i) gt 3000 and ord(i) le 3500);
test(i) = yes$(ord(i) gt 3500);

To generate this classifier a quadratic optimization model is solved, using only the data in the training set:
$$
  \begin{array}{lc}
    \min_{w,\gamma,\psi} & \frac{1}{2} w'w + C \sum_i \psi_i \\
    \text{s.t.} & y_i [w'x_i + \gamma] \geq 1 - \psi_i, i=1,\ldots,m \\
    & \psi_i \geq 0
  \end{array}
$$
Here $m$ runs over the set of training examples, and $y_i$ is $1$ if $i \in P_+$ and $-1$ if $i \in P_-$.  $x_i$ are the values of the predictor variables.
Note that C is a tradeoff parameter between errors $\psi_i$ and a measure of generalization ability $w$ and we will choose an appropriate value of C.  Initially set $C=1$.
  
The dual problem of this quadratic program is:
$$
  \begin{array}{lc}
    \max_{\alpha} & \sum_i \alpha_i 
    - \frac{1}{2} \sum_{i,j} \alpha_i \alpha_j y_i y_j x_i' x_j \\
    \text{s.t.} & \sum_i y_i \alpha_i = 0 \\
    & C \geq \alpha_i \geq 0
  \end{array}
$$

### You should formulate both these problems in GAMS.  

Note that for the
  dual problem, you may want to introduce some intermediate variables 
$$
    v_k = \sum_i \alpha_i y_i x_{ik}
$$
  and express the objective function as
$$
  \sum_i \alpha_i 
  - \frac{1}{2} \sum_{k} v_k^2
$$
  where $k$ runs over the predictor features.  

In [3]:
%%gams
# write gams models here
parameter C "a tradeoff parameter";
* initialize C to be 1
C = 1;

* dual problem
free variable z_dual;
variable v(headr), a(i);

* set the upper and lower bounds for a(i)
a.lo(i)$train(i) = 0;
a.up(i)$train(i) = C;

equations dual_objective
          def_v
          dual_constr1;
          
* objective function
dual_objective..
    z_dual =e= sum(i$train(i), a(i)) - 0.5*sum(headr$x(headr), sqr(v(headr)));
    
* equation definitions
def_v(headr)$x(headr)..
    v(headr) =e= sum(i$train(i), a(i)*y(i)*Data(i,headr));

dual_constr1..
    sum(i$train(i), a(i)*y(i)) =e= 0 ;

model svmdual /dual_objective,def_v,dual_constr1/;
option optcr=0;
solve svmdual using qcp max z_dual;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1534.0031,12,3011,QCP,CONOPT,0.297


### Solve (the easiest one) using the qcp solver of your choice, and the use the multiplier information from this first solve to set the starting values for the second solve.  
Try to make the second solve take as little time as possible.   

In [4]:
%%gams
# write gams code that solves one model here
# then uses that solution to provide a (very good) starting point for the other model solve
free variable z_primal;
variable w(headr) "coeffcients of independent variables"
         psi(i) "error term of each training sample"
         gamma "a constant variable";

w.fx("Rings") = 0; 

* let psi to be positive
psi.lo(i)$train(i) = 0;

* multiplier information from dual problem
parameter psi_value(i), gamma_value;
psi_value(train) = a.m(train);
gamma_value = dual_constr1.m;

equations primal_objective, primal_constr;

* objective function for primal problem
primal_objective..
    z_primal =e= sum(headr, 0.5*sqr(w(headr))) + C*sum(i$train(i), psi(i));

primal_constr(i)$train(i)..
    y(i)*sum(headr$x(headr), w(headr)*Data(i,headr)) + y(i)*gamma =g= 1 - psi(i);

* use the multiplier information from first solve to set the starting values
psi.l(train) = psi_value(train);
gamma.l = gamma_value;

model svmprimal /primal_objective,primal_constr/;
option optcr = 0;
solve svmprimal using qcp min z_primal;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),1534.0031,3001,3013,QCP,CONOPT,0.473


Write code that experiments with multiple values of C and evaluate the number of errors 
you make on the tuning set.   Using a limited amount of computation (10 minutes at most), determine a good value for C and report it in the notebook.  

In [5]:
%%gams
# write gams code here to calculate the value of C

* use 15 iterations
set iters /1*15/;
* set of values for each iteration
parameters values(iters) /1 0.005, 2 0.05, 3 0.1, 4 0.5, 5 1,
                          6 5, 7 10, 8 50, 9 80, 10 100,
                          11 500, 12 1000, 13 1500, 14 2000, 15 5000/;

set i_pred(i);
i_pred(i)$(tuning(i)) = yes;                    
parameter y_pred(i), num_error;

parameter d(*);
loop (iters,
  C = values(iters);
  solve svmprimal using qcp min z_primal;
  y_pred(i_pred) = -1;
  y_pred(i_pred)$(sum(x, w.l(x) * Data(i_pred, x)) + gamma.l > 0) = 1;
  num_error = sum(i_pred, 1$(y_pred(i_pred) <> y(i_pred)));
  d(iters) = num_error;
);

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),9.913900e+00,3001,3013,QCP,CONOPT,0.457
1,Normal (1),OptimalLocal (2),9.167220e+01,3001,3013,QCP,CONOPT,0.171
2,Normal (1),OptimalLocal (2),1.773609e+02,3001,3013,QCP,CONOPT,0.111
3,Normal (1),OptimalLocal (2),7.986040e+02,3001,3013,QCP,CONOPT,0.146
4,Normal (1),OptimalLocal (2),1.534003e+03,3001,3013,QCP,CONOPT,0.066
5,Normal (1),OptimalLocal (2),7.256767e+03,3001,3013,QCP,CONOPT,0.098
6,Normal (1),OptimalLocal (2),1.436090e+04,3001,3013,QCP,CONOPT,0.046
7,Normal (1),OptimalLocal (2),7.109136e+04,3001,3013,QCP,CONOPT,0.036
8,Normal (1),OptimalLocal (2),1.136267e+05,3001,3013,QCP,CONOPT,0.015
9,Normal (1),OptimalLocal (2),1.419830e+05,3001,3013,QCP,CONOPT,0.007


In [6]:
%gams_pull -d values d
# print out the best value of C which is the C that results in the lowest number of errors
print("best C:{}".format(values.iloc[d['value'].idxmin()]['value']))
# display the number of errors for each iteration
display(d)

best C:10.0


,*,value
0,1,214.0
1,2,167.0
2,3,156.0
3,4,143.0
4,5,140.0
5,6,142.0
6,7,138.0
7,8,139.0
8,9,139.0
9,10,139.0


Now with the fixed value of C from the above, determine the error rate that you would expect to make on unseen data (this is the first time you should use the test data, and you should not solve any optimization problem with that data, just see if your prediction is correct or not).

In [7]:
%%gams
# write gams code to simply evaluate the errorrate in your prediction on the test set.
set i_pred(i);
i_pred(i)$(test(i)) = yes;                    
parameter y_pred(i), errorrate;

* plug in the optimal value of C, which is C=10
C = 10;

solve svmprimal using qcp min z_primal;
y_pred(i_pred) = -1;
y_pred(i_pred)$(sum(x, w.l(x) * Data(i_pred, x)) + gamma.l > 0) = 1;
errorrate = sum(i_pred, 1$(y_pred(i_pred) <> y(i_pred)))/card(i_pred);

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),OptimalLocal (2),14360.8971,3001,3013,QCP,CONOPT,0.062


In [8]:
%gams_pull -d errorrate
display(errorrate)

,value
0,0.24554
